<a href="https://colab.research.google.com/github/quantaosun/Waterswap_example/blob/main/waterswap_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 药物分子-溶剂水交换结合自由能分子动力学模拟

这是一个用来计算一个蛋白质靶点和一个小分子药物之间结合自由能的笔记本，使用的是英国的开源软件Sire的一个模块，名字为Waterswap，如果你希望理解背后的逻辑，你需要仔细阅读下面的两篇文献。


好消息是，即使你使用个人笔记本或者配置较低的服务器，你可以把你的任务无限次的延续，只要你有耐心，总会完成。而计算任务会自动从上次你结束的地方继续计算，你无需做任何其他设置。

如果你有许多CPU比如超过50块，那么你的计算可能会在一到两天内完成。

Woods, C. J., Malaisree, M., Hannongbua, S., Mulholland, A.J., “A water-swap reaction coordinate for the calculation of absolute protein-ligand binding free energies”, J. Chem. Phys. 134, 054114, 2011, DOI:10.1063/1.3519057 

Woods, C. J., Malaisree, M., Michel, J., Long, B., McIntosh-Smith, S., Mulholland, A. J., “Rapid Decomposition and Visualisation of Protein-Ligand Binding Free Energies by Residue and by Water”, Faraday Discussions 169: Molecular Simulation and Visualisation, 2014, DOI:10.1039/C3FD00125C



方法点评： 这个方法根据个人的使用感受（github/quantaosun），原理上不如FEP，但是要好于MMPBSA。但FEP对硬件的苛刻要求和繁琐的使用技巧，后者昂贵的价格，往往让人望而却步。

这个方法是完全免费的，唯一的硬伤也许就是速度慢一些，并且跟MMPBSA一样结果的数值区间是不能落入真实的实验区间的，但两个化合物直接的差值是具有可比性的。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Install Omnia
################################################################################
# INSTALL CONDA ON GOOGLE COLAB
################################################################################
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh 
! chmod +x Miniconda3-py37_4.12.0-Linux-x86_64.sh 
! bash ./Miniconda3-py37_4.12.0-Linux-x86_64.sh  -b -f -p /usr/local 
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
#安装sire
!yes | conda install -c conda-forge -c omnia -c michellab sire

In [ ]:
#@title ABF waterswap validation
!optimise_openmm
!waterswap -h

## Overwrite the SYS_gaff2.prmtop 
This is a temperary solution to an error. This error should be automaticcly resolved when the new 2023 version of watermap update in the near future.

In [ ]:
!mv SYS_gaff2.prmtop SYS_gaff2_temp.prmtop

In [ ]:
!grep -v %COMMENT SYS_gaff2_temp.prmtop > SYS_gaff2.prmtop

In [4]:
! grep -v %COMMENT /content/SYS_gaff2.prmtop > SYS.prmtop

即使因为这样那样的原因没有完成，你也完全不必担心，比如任务跑到了250步骤，则下次重复运行时，会在动从251步开始，你也许都察觉不到这背后跟第一次运行时的区别。

In [ ]:
!waterswap -l LIG  -t SYS.prmtop -c SYS_gaff2.crd -n 1000

请至少跑到1000步，再运行分析指令。具体的原理请参考以上两篇文献。

In [ ]:
#Analysi
!analyse_freenrg -i output/freenrgs.s3 -o results.txt

误差评估： 结果会给出三个结果，如果你的模拟比较好，这三个数值应当接近。